In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Here we create our own synthetic dataset suitable for regression

In [3]:
# Generate synthetic dataset
X, y = make_regression(n_samples=1000, n_features=5, noise=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
X_train

array([[-0.25651761,  0.80681276,  0.57035581,  1.6217852 , -1.38594098],
       [-1.33457381, -1.41201932,  1.67176971, -0.99412132, -0.24814663],
       [ 0.81674884, -0.36086142, -0.473323  , -1.33530054,  2.18750092],
       ...,
       [-0.05263683, -0.27627953, -1.70696913,  0.77997602,  0.74116332],
       [ 0.48844163,  1.94568126,  0.65716814, -0.0025513 ,  0.02259101],
       [-0.88802148, -0.49275216,  1.82706062,  0.86860715,  1.51793406]])

In [5]:
y_train

array([ 1.96954537e+02, -4.16530841e+01, -1.00494188e+02,  2.16744489e+02,
       -2.68065934e+02,  2.36795384e+01,  2.13327253e+02,  1.83182675e+02,
        1.63073735e+02, -1.95326398e+00,  8.29139140e+01,  2.82716089e+02,
       -1.44077025e+02, -3.02311628e+02,  9.89543477e+00,  2.50287459e+01,
        1.30715661e+02,  4.73259246e+01,  1.16056070e+02, -5.56195600e+01,
       -8.68517028e+01, -1.78303411e+02,  7.21587373e+01,  1.34576389e+01,
       -5.69875016e+01,  2.17556258e+02,  7.17379725e+00, -1.06298258e+02,
       -2.47730147e+02, -4.11209283e+01, -1.06679027e+01, -2.70208377e+02,
        1.07319202e+02,  8.95377893e+01,  2.04449635e+02, -2.23410833e-02,
       -4.27169107e+01, -2.94929051e+02, -5.07578320e+01,  7.33960423e+01,
        1.33341344e+02,  4.00599250e+02, -2.87789250e+02,  2.37560535e+02,
        1.64003300e+01, -8.40413723e+01, -3.14628503e+02,  1.13706123e+02,
       -1.82771057e+02, -2.55170152e+01, -1.79514978e+02,  2.18602477e+01,
       -2.97757320e+02,  

In [6]:
# Scale the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

In [7]:
X_train

array([[-0.3010919 ,  0.81184809,  0.57352972,  1.63690618, -1.38627851],
       [-1.4142542 , -1.37930807,  1.68578131, -1.03637411, -0.29760146],
       [ 0.80712468, -0.34126147, -0.48041867, -1.3850363 ,  2.0329019 ],
       ...,
       [-0.0905719 , -0.25773458, -1.72620363,  0.77663389,  0.64900108],
       [ 0.46812639,  1.93651127,  0.66119627, -0.02305633, -0.03855126],
       [-0.95316022, -0.47150711,  1.84260025,  0.86720894,  1.39223949]])

In [8]:
y_train

array([ 1.29715949e+00, -3.11493219e-01, -7.08190107e-01,  1.43058004e+00,
       -1.83793082e+00,  1.28968404e-01,  1.40754160e+00,  1.20431190e+00,
        1.06874078e+00, -4.38436665e-02,  5.28316665e-01,  1.87534954e+00,
       -1.00201831e+00, -2.06880956e+00,  3.60382804e-02,  1.38064535e-01,
        8.50588052e-01,  2.88388388e-01,  7.51755543e-01, -4.05652866e-01,
       -6.16214754e-01, -1.23276689e+00,  4.55807061e-01,  6.00540653e-02,
       -4.14875300e-01,  1.43605285e+00,  1.76894551e-02, -7.47320177e-01,
       -1.70083034e+00, -3.07905514e-01, -1.02596304e-01, -1.85237481e+00,
        6.92853041e-01,  5.72973726e-01,  1.34769019e+00, -3.08257070e-02,
       -3.18665361e-01, -2.01903747e+00, -3.72875908e-01,  4.64148765e-01,
        8.68289974e-01,  2.67009811e+00, -1.97090212e+00,  1.57091834e+00,
        7.98931967e-02, -5.97267975e-01, -2.15184788e+00,  7.35912596e-01,
       -1.26288701e+00, -2.02706540e-01, -1.24093507e+00,  1.16703050e-01,
       -2.03810518e+00,  

### We build the forward model with linear layers

In [9]:
# Define the forward model
class ForwardModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ForwardModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_dim)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
# Initialize and train the model
input_dim = X_train.shape[1]
output_dim = 1
model = ForwardModel(input_dim, output_dim)

In [11]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [13]:
y_train.shape

(800,)

In [14]:
y_train_tensor.shape

torch.Size([800, 1])

### Training and testing the model

In [15]:
# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.0030
Epoch [200/1000], Loss: 0.0012
Epoch [300/1000], Loss: 0.0007
Epoch [400/1000], Loss: 0.0005
Epoch [500/1000], Loss: 0.0004
Epoch [600/1000], Loss: 0.0003
Epoch [700/1000], Loss: 0.0003
Epoch [800/1000], Loss: 0.0002
Epoch [900/1000], Loss: 0.0002
Epoch [1000/1000], Loss: 0.0002


In [16]:
# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 0.0006


### Building the inverse model using gradient descent

In [17]:
# Function to perform inverse modeling
def inverse_model(model, target, initial_input, num_iterations=1000, learning_rate=0.01):
    # Convert target to tensor
    target_tensor = torch.tensor(target, dtype=torch.float32).view(-1, 1)
    
    # Initialize input as a tensor with requires_grad=True
    input_tensor = torch.tensor(initial_input, dtype=torch.float32, requires_grad=True)
    
    # Define optimizer to adjust the input
    optimizer = optim.Adam([input_tensor], lr=learning_rate)
    
    # Iterative optimization
    for i in range(num_iterations):
        optimizer.zero_grad()
        output = model(input_tensor)
        loss = criterion(output, target_tensor)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Iteration [{i+1}/{num_iterations}], Loss: {loss.item():.4f}')
    
    # Return the adjusted input
    return input_tensor.detach().numpy()

### Building the inverse model using genetic algorithms

In [18]:
def inverse_model_ga(model, target, initial_input, num_generations=1000, pop_size=100, mutation_rate=0.01):
    # Convert target to tensor
    target_tensor = torch.tensor(target, dtype=torch.float32).view(-1, 1)

    # Initialize population as a 2D array with random inputs
    population = np.random.normal(loc=initial_input, scale=0.1, size=(pop_size, len(initial_input)))

    for generation in range(num_generations):
        # Compute fitness for each input in the population
        fitness = np.empty(pop_size)
        for i in range(pop_size):
            input_tensor = torch.tensor(population[i], dtype=torch.float32)
            output = model(input_tensor)
            fitness[i] = -criterion(output, target_tensor).item()

        # Select the best inputs based on their fitness
        selected_indices = np.argsort(fitness)[-pop_size//2:]
        selected_population = population[selected_indices]

        # Generate the next generation by applying crossover and mutation
        next_population = np.zeros_like(population)
        for i in range(pop_size):
            if np.random.rand() < mutation_rate:
                # Mutation: add small random noise to one of the selected inputs
                next_population[i] = selected_population[np.random.randint(pop_size//2)] + np.random.normal(scale=0.01, size=len(initial_input))
            else:
                # Crossover: randomly combine parts of two selected inputs
                parent1, parent2 = selected_population[np.random.choice(pop_size//2, 2, replace=False)]
                crossover_point = np.random.randint(len(initial_input))
                next_population[i] = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])

        population = next_population

        if (generation+1) % 100 == 0:
            print(f'Generation [{generation+1}/{num_generations}], Best Fitness: {-fitness[selected_indices[-1]]:.4f}')

    # Return the best input found
    best_input = population[np.argmax(fitness)]
    return best_input

In [19]:
# Desired target value (scaled)
desired_target = scaler_y.transform([[200]])  # Example target
desired_target

array([[1.31769149]])

In [20]:
# Initial input (random or mean of the training data)
initial_input = torch.mean(X_train_tensor, dim=0).numpy()  # Example initial input

In [21]:
initial_input

array([-5.9604643e-10,  4.1723252e-09, -1.1920929e-09, -1.1920929e-09,
        2.3841857e-09], dtype=float32)

In [22]:
X_train_tensor.max()

tensor(3.2066)

In [23]:
# Perform inverse modeling
adjusted_input = inverse_model(model, desired_target, initial_input)

Iteration [100/1000], Loss: 0.0036
Iteration [200/1000], Loss: 0.0000
Iteration [300/1000], Loss: 0.0000
Iteration [400/1000], Loss: 0.0000


/Users/surya/anaconda3/envs/Tf-metal/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Iteration [500/1000], Loss: 0.0000
Iteration [600/1000], Loss: 0.0000
Iteration [700/1000], Loss: 0.0000
Iteration [800/1000], Loss: 0.0000
Iteration [900/1000], Loss: 0.0000
Iteration [1000/1000], Loss: 0.0000


In [24]:
adjusted_input

array([0.5974772 , 0.6630324 , 0.66031426, 0.68594354, 0.6782127 ],
      dtype=float32)

In [25]:
adjusted_input_2D = adjusted_input.reshape(1, -1)  # Reshape to 2D array
adjusted_input_original_scale = scaler_X.inverse_transform(adjusted_input_2D)
print("Adjusted Input (Original Scale):", adjusted_input_original_scale)

Adjusted Input (Original Scale): [[0.613713   0.65611744 0.65629476 0.691232   0.7716929 ]]


In [26]:
model(torch.tensor(adjusted_input))

tensor([1.3177], grad_fn=<ViewBackward0>)

In [27]:
adjusted_input_ga = inverse_model_ga(model, desired_target, initial_input)

/Users/surya/anaconda3/envs/Tf-metal/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Generation [100/1000], Best Fitness: 0.4575
Generation [200/1000], Best Fitness: 0.1982
Generation [300/1000], Best Fitness: 0.0590
Generation [400/1000], Best Fitness: 0.0007
Generation [500/1000], Best Fitness: 0.0000
Generation [600/1000], Best Fitness: 0.0000
Generation [700/1000], Best Fitness: 0.0000
Generation [800/1000], Best Fitness: 0.0000
Generation [900/1000], Best Fitness: 0.0000
Generation [1000/1000], Best Fitness: 0.0000


In [28]:
adjusted_input

array([0.5974772 , 0.6630324 , 0.66031426, 0.68594354, 0.6782127 ],
      dtype=float32)

In [29]:
adjusted_input_ga

array([0.58259186, 0.58373835, 0.75857578, 0.68010365, 0.5238336 ])

In [30]:
model(torch.tensor(adjusted_input))

tensor([1.3177], grad_fn=<ViewBackward0>)

In [31]:
model(torch.tensor(adjusted_input_ga,dtype=torch.float32))

tensor([1.3177], grad_fn=<ViewBackward0>)

In [5]:
df = pd.read_csv('Regression_dataset.csv')
df

,Unnamed: 0,0,1,2,3,4,5
0,0,1.060134,-0.546352,-0.205450,1.209442,0.036726,136.904764
1,1,-0.844341,-0.091294,1.724862,1.153446,-1.234914,156.421240
2,2,-0.094526,1.155948,-0.428693,-0.392627,-0.500839,-59.967029
3,3,0.489770,-0.739551,0.027801,-0.882449,0.964301,-50.583449
4,4,0.601500,1.873315,0.462507,-0.635150,0.010306,19.082291
...,...,...,...,...,...,...,...
995,995,-0.602977,-1.817764,-1.043620,-0.468633,0.303158,-149.023696
996,996,-1.623561,-0.233480,-0.519278,0.081507,0.947481,-88.531015
997,997,0.854356,-0.642524,-0.126632,-0.502459,-0.674379,-29.607573
998,998,-1.680343,-0.302481,-0.366340,-1.125071,0.713336,-192.580530


In [7]:
df.columns[:-1]

Index(['Unnamed: 0', '0', '1', '2', '3', '4'], dtype='object')

In [8]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.9 MB/s eta 0:00:0000:0100:010m
  Preparing metadata (setup.py) ... done
  Created wheel for deap: filename=deap-1.4.1-cp310-cp310-macosx_11_0_arm64.whl size=103918 sha256=8fe2c028fb5e307761615b8a7a99f863010615411e03f16ca6691a68a3cce3f0
  Stored in directory: /Users/surya/Library/Caches/pip/wheels/0e/f2/e3/f1bd8b40cf7eefa319d5a27aa9ba3cc588e6ada69b5919590d
Successfully built deap
